# VAE with Newton's law decoder

In [1]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gpytorch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import copy
from torch.utils.data import Dataset, DataLoader,random_split, Subset
from tqdm import tqdm
import random

sys.path.append(os.path.abspath('..'))
from Models.PIVAE import PIVAE
from physics import NewtonsLaw

In [2]:
def set_seed_(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# Import RICO data 
file_path = '../Data/RICO4_Dataset_processed.hdf'
df = pd.read_hdf(file_path)
#  define surrounding temperature
df.loc[:, 'sur_temp'] = (df['RTD417'] + df['B.ASTRHT2.T'])/2
df_c = df.copy()
df_c = df[df['temp_change_class'] == 'c']
def select_points(group):
    return group.iloc[::10]  # Select every 10th row
df_c = df_c.groupby('Scheduler Step').apply(select_points).reset_index(drop=True)
df_c = df_c.dropna()

/var/folders/46/30710m094qv2_dhfs08nzzd00000gn/T/ipykernel_16754/3618831305.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_c = df_c.groupby('Scheduler Step').apply(select_points).reset_index(drop=True)


In [4]:
class RICO4Dataset(Dataset):
    def __init__(self, df, select_var):
        self.df = df
        self.select_var = select_var
        self.unique_batches = df['interval'].unique()

    def __len__(self):
        return len(self.unique_batches)

    def __getitem__(self, idx):
        batch = self.unique_batches[idx]
        temp = self.df[self.df['interval'] == batch][self.select_var]
        
        # Normalize and convert to tensors
        ti = torch.tensor(temp['time_within_interval'].to_numpy() / 240, dtype=torch.float32)
        Ts = torch.tensor(temp['sur_temp'].to_numpy(), dtype=torch.float32)
        heat_obs = torch.tensor(temp['B.RTD1'].to_numpy(), dtype=torch.float32)
        
        # Stack the tensors to create the input batch
        x_batch = torch.stack((heat_obs, Ts, ti), dim=1)
        
        return x_batch
# Initialize the dataset and train_loader
select_var = ['time_within_interval', 'sur_temp', 'B.RTD1']
dataset = RICO4Dataset(df_c,select_var)
dataloader = DataLoader(dataset)
# Define split ratios
train_ratio = 0.7
test_ratio = 1-train_ratio

# Calculate split sizes
total_size = len(dataset)
train_size = int(train_ratio * total_size)
test_size = total_size - train_size

seed = 0
generator = torch.Generator().manual_seed(seed)

# Split the dataset
train_subset, test_subset = random_split(
    dataset,
    [train_size, test_size],
    generator=generator
)
train_loader = DataLoader(train_subset, batch_size=1, shuffle=False)
val_loader = DataLoader(test_subset, batch_size=1, shuffle=False)

In [ ]:
# Set the variables to be selected from the dataframe
set_seed_(123)
model = PIVAE(hidden_layers=[10,10], 
                 activation=nn.Tanh(),
                 mu_prior=torch.tensor(0),
                 var_prior=torch.tensor(2.0))
# Initialize model parameters and optimizer
pars = model.parameters()
opt = torch.optim.Adam(pars, lr=0.0001)

# Lists to store loss values
l_loss, KL, mse = [], [], []

# Set beta value and number of epochs
beta_phy = 1
num_epochs = 400
l_loss, KL, mse = model.fit(train_loader, opt, beta_phy, num_epochs)


Progress: Epoch 80/400 | Loss=1.9958, KL=0.1593, MSE=1.8365
Progress: Epoch 160/400 | Loss=2.0581, KL=0.1757, MSE=1.8823
Progress: Epoch 240/400 | Loss=2.2261, KL=0.1870, MSE=2.0391
Progress: Epoch 320/400 | Loss=2.2910, KL=0.1946, MSE=2.0965
Progress: Epoch 400/400 | Loss=1.1676, KL=0.1920, MSE=0.9757


In [6]:
torch.save(model.state_dict(), 'pre_trained_models/PIVAE_cooling.pth')